In [65]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

from io import StringIO
import os

In [66]:
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [67]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

In [68]:
s3 = boto3.resource('s3')
s3

s3.ServiceResource()

In [69]:
bucket = s3.Bucket(name="noaa-global-hourly-pds")
bucket

s3.Bucket(name='open-data-noaa')

In [70]:
#Get File Name List
prefix = "1970/"
#prefix = "2010/"
#prefix = "2005/"
#prefix = "2000/"
#prefix = "1995/"
#prefix = "1990/"
#prefix = "1985/"
#prefix = "1980/"
#prefix = "1975/"
#prefix = "1970/"
FilesNotFound = True
FileNameList=[]
for obj in bucket.objects.filter(Prefix=prefix):
    FileNameList.append('{0}{1}/{2}'.format('s3://',bucket.name, obj.key))
    #print('{0}/{1}'.format(bucket.name, obj.key))
    FilesNotFound = False
if FilesNotFound:
     print("ALERT", "No file in {0}/{1}".format(bucket, prefix))


In [ ]:
FileNameList

In [104]:
#import random
#desired_pct = 1.65
#random.seed(20)
#file_sample = random.sample(FileNameList, int(len(FileNameList) * desired_pct / 100))

In [77]:
len(FileNameList)

3006

In [49]:
import random
desired_pct = 20
random.seed(20)
file_sample = random.sample(FileNameList, int(len(FileNameList) * desired_pct / 100))

In [50]:
len(file_sample)

601

## Merge Data

In [80]:
from py4j.protocol import Py4JJavaError

def file_exist(sc, file):
    try:
        rdd = sc.textFile(file)
        rdd.take(1)
        return True
    except Py4JJavaError as e:
        return False

In [83]:
#Merge
rdd = sc.textFile(FileNameList[0])
i=0
rdd_all=rdd
rdd.count()

2498

In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
rdd3 = sc.parallelize([7, 8, 9])

rdd = sc.union([rdd1, rdd2, rdd3])

In [ ]:
RDDs = sc.parallelize([["panda", 0], ["pink", 3]])
RDDList = []
for i in range(0,len(RDDs.first())):    
    RDDList.append(RDDs.map(lambda x: [x[i]]).collect())

print (RDDList[0])
print (RDDList[1])

In [84]:
while i < len(FileNameList)/7:
    if file_exist(sc, FileNameList[i]):
        #print(FileNameList[i])
        rdd_sep=sc.textFile(FileNameList[i])
        rdd_all=rdd_all.union(rdd_sep)
        
    else:
        print('File does not exist, skipping: ' + FileNameList[i])
    i += 1
rdd_all.count()

1193453

In [ ]:
rdd_all.count()

In [ ]:
  
rdd_full=sc.union([rdd_all, rdd_all2])    
rdd_all.count()

In [27]:
#Save
rdd_all.repartition(1).saveAsTextFile("s3://open-data-noaa/1970sample_v2")

In [28]:
df=spark.read.format("csv").option("header", "true").load("s3://open-data-noaa/1970sample_v2")

In [29]:
df.head(5)

[Row(STATION='99999993729', DATE='1970-01-01T00:00:00', SOURCE='E', LATITUDE='35.2326', LONGITUDE='-75.6219', ELEVATION='3.4', NAME='CAPE HATTERAS BILLY MITCHELL FIELD, NC US', REPORT_TYPE='SAO  ', CALL_SIGN='HAT  ', QUALITY_CONTROL='V020', WND='290,5,N,0057,5', CIG='22000,5,9,N', VIS='016000,5,9,9', TMP='+0128,4', DEW='+0061,5', SLP='10139,4', AA1='01,0000,9,5', GA1='99,9,+99999,9,00,4', GA2='99,9,+99999,9,99,9', GA3='99,9,+99999,9,99,9', GA4='99,9,+99999,9,99,9', GD1='4,08,5,+99999,9,9', GD2='0,00,5,+99999,9,9', GD3='0,00,5,+99999,9,9', GD4='0,00,5,+99999,9,9', GF1='08,06,5,99,9,99,9,99999,9,99,9,99,9', MA1='99999,9,10135,4', MW1=None, MW2=None, MW3=None, EQD='N01 00000JPWTH 1'),
 Row(STATION='99999993729', DATE='1970-01-01T03:00:00', SOURCE='E', LATITUDE='35.2326', LONGITUDE='-75.6219', ELEVATION='3.4', NAME='CAPE HATTERAS BILLY MITCHELL FIELD, NC US', REPORT_TYPE='SAO  ', CALL_SIGN='HAT  ', QUALITY_CONTROL='V020', WND='060,5,N,0077,5', CIG='22000,5,9,N', VIS='016000,5,9,9', TMP='+0

In [30]:
df.write.csv("s3://open-data-noaa/1970samples_v2", sep=',', header=True,mode="overwrite")

In [111]:
sc.stop()